In [1]:
from UTILITY_quickstart import *

In [2]:
#Load a default configuration
importedDefaultSettings = loadConfig("setLattice_configs/defaults.yml")

#Initialize tao object
tao = initializeTao(
    #inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
    #numMacroParticles = 1e4,
    #csrTF = False
)

#Set lattice to the loaded config
setLattice(tao, **importedDefaultSettings)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
WARNING! No beam file is specified!
Number of macro particles defined by input file


## Get all transport matrices

In [3]:
listOfBPMs = ["M1FF", "M2FF", "M3FF", "M4FF", "M5FF", "M0EX", "M1EX", "M2EX", "M3EX"]

allMatrices = { bpm : getMatrix(tao, "PATCH_SECTOR20", bpm) for bpm in listOfBPMs }

In [4]:
#Apply random offsets from nominal
tao.cmd("set ele patch_sector20 x_offset = 3.1e-6")
tao.cmd("set ele patch_sector20 x_pitch =  6.6e-6")
tao.cmd("set ele patch_sector20 y_offset = 8.2e-6")
tao.cmd("set ele patch_sector20 y_pitch =  6.1e-6")
tao.cmd(f"set ele patch_sector20 e_tot_set = {10e9 * ( 1 + 0.027 ) }")


setLattice(tao, **importedDefaultSettings)

## Solve for x and x'

In [5]:
measuredXValues = { bpm : -1 * tao.ele_orbit(bpm)["x"] for bpm in listOfBPMs } #Can't remember why this has to be negative

assembledMatrix = np.array([ m[0] for m in allMatrices.values() ])

inverseMatrix = np.linalg.pinv(assembledMatrix)

solution = np.dot(inverseMatrix, list(measuredXValues.values()))

print(f"x = {1e6 * solution[0]} um, x' = {1e6 * solution[1]} urad")

x = 3.1000056561218 um, x' = 6.599997493685794 urad


## Solve for y, y', and energy

In [6]:
measuredYValues = { bpm : -1 * tao.ele_orbit(bpm)["y"] for bpm in listOfBPMs } #Can't remember why this has to be negative

assembledMatrix = np.array([ m[2] for m in allMatrices.values() ])

inverseMatrix = np.linalg.pinv(assembledMatrix)

solution = np.dot(inverseMatrix, list(measuredYValues.values()))

print(f"y = {1e6 * solution[2]} um, y' = {1e6 * solution[3]} urad, energy offset = {100 * solution[5]}%")

y = 8.200000017094256 um, y' = 6.100000005324288 urad, energy offset = 2.7160701721103306%
